In [10]:
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import balanced_accuracy_score, make_scorer
from imblearn.over_sampling import RandomOverSampler
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.cluster import SpectralClustering
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import plot_tree


#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sys.path.insert(0, '/Users/gracewang/Documents/GitHub/elecfinal')
sys.path.insert(0, 'D:\Fall23 Coursework\ELEC478\Competition\elecfinal')
from ml_pipeline import train_n_predict, validation, clean_split
from Data.data_cleaner import cleaner

In [11]:
## Clean and split data
train_path = "../Data/train_data.csv"
feature_path = "../Data/feature_weights.csv"
morph_path = "../Data/imputed_morph_embed.csv"
clean_data = cleaner(train_path, feature_path, morph_path)

In [12]:
clean_data

,ID,axonal_coor_x,axonal_coor_y,axonal_coor_z,dendritic_coor_x,dendritic_coor_y,dendritic_coor_z,adp_dist,post_skeletal_distance_to_soma,pre_skeletal_distance_to_soma,...,AL_pre,V1_pre,RL_post,V1_post,AL_post,area1,area2,area3,minicol_dist,nuclei_adp_dist
0,42593,0.445791,-1.742979,2.174696,0.444810,-1.740997,2.176315,-1.515697,2.007164,0.931055,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.076496
1,42594,0.507204,0.365932,0.044530,0.507420,0.368718,0.052049,-1.225578,1.070301,0.268030,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,-0.306546
2,42595,0.460781,-1.807635,2.215744,0.449166,-1.817435,2.210853,0.632301,2.099967,0.905442,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.118612
3,42596,0.433668,-1.686017,2.114604,0.442015,-1.682527,2.140593,0.645635,1.931656,0.956222,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.035778
4,42597,0.451199,0.293911,0.745737,0.449891,0.328971,0.733044,1.334344,1.668055,-0.042977,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,-0.587712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185827,228420,0.709191,-0.639091,0.441756,0.717718,-0.655646,0.436799,0.452526,-0.282486,0.324338,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.214541,-0.780397
185828,228421,0.543500,-1.794867,0.474081,0.537437,-1.818791,0.460571,0.687832,-0.042144,0.817518,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.047859
185829,228422,0.577400,-2.030573,-0.272713,0.566982,-2.048065,-0.249929,0.987837,0.182085,0.910021,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.385538
185830,228423,0.684728,-1.930889,-0.146313,0.698481,-1.913531,-0.140918,1.162151,-0.079219,0.792450,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.261090


In [13]:
import random

def splitter(data):
# perform stratified sampling of pre-synaptic neurons
    pre_nucleus_ids = pd.unique(data["pre_nucleus_id"])
    #print(len(pre_nucleus_ids))

    # Use 80% of the pre-nucleus ids
    #print(len(pre_nucleus_ids))
    train_nucleus_idx = random.sample(range(0, len(pre_nucleus_ids)), int(np.floor(0.8*len(pre_nucleus_ids))))
    train_nucleus_ids = pre_nucleus_ids[train_nucleus_idx]
    training = data[data["pre_nucleus_id"].isin(train_nucleus_ids)]
    X_train = training.drop(columns='connected')
    y_train = training['connected']
    pre_nucleus_ids = np.delete(pre_nucleus_ids, train_nucleus_idx)

    # # Use 20% for query set
    # #print(len(pre_nucleus_ids))
    # query_nucleus_idx = random.sample(range(0, len(pre_nucleus_ids)), int(np.floor(0.5*len(pre_nucleus_ids))))
    # query_nucleus_ids = pre_nucleus_ids[query_nucleus_idx]
    # query = data[data["pre_nucleus_id"].isin(query_nucleus_ids)]
    # X_query = query.drop(columns='connected')
    # y_query = query['connected']
    # pre_nucleus_ids = np.delete(pre_nucleus_ids, query_nucleus_idx)


    # Use 20% for validation
    #print(len(pre_nucleus_ids))
    validation = data[data["pre_nucleus_id"].isin(pre_nucleus_ids)]
    X_val = validation.drop(columns='connected')
    y_val = validation['connected']

    return X_train, X_val, y_train, y_val

In [14]:
random.seed(1)
X_train, X_val, y_train, y_val = splitter(clean_data)
X_train

,ID,axonal_coor_x,axonal_coor_y,axonal_coor_z,dendritic_coor_x,dendritic_coor_y,dendritic_coor_z,adp_dist,post_skeletal_distance_to_soma,pre_skeletal_distance_to_soma,...,AL_pre,V1_pre,RL_post,V1_post,AL_post,area1,area2,area3,minicol_dist,nuclei_adp_dist
0,42593,0.445791,-1.742979,2.174696,0.444810,-1.740997,2.176315,-1.515697,2.007164,0.931055,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.076496
1,42594,0.507204,0.365932,0.044530,0.507420,0.368718,0.052049,-1.225578,1.070301,0.268030,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,-0.306546
2,42595,0.460781,-1.807635,2.215744,0.449166,-1.817435,2.210853,0.632301,2.099967,0.905442,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.118612
3,42596,0.433668,-1.686017,2.114604,0.442015,-1.682527,2.140593,0.645635,1.931656,0.956222,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.035778
4,42597,0.451199,0.293911,0.745737,0.449891,0.328971,0.733044,1.334344,1.668055,-0.042977,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,-0.587712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185827,228420,0.709191,-0.639091,0.441756,0.717718,-0.655646,0.436799,0.452526,-0.282486,0.324338,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.214541,-0.780397
185828,228421,0.543500,-1.794867,0.474081,0.537437,-1.818791,0.460571,0.687832,-0.042144,0.817518,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.047859
185829,228422,0.577400,-2.030573,-0.272713,0.566982,-2.048065,-0.249929,0.987837,0.182085,0.910021,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.385538
185830,228423,0.684728,-1.930889,-0.146313,0.698481,-1.913531,-0.140918,1.162151,-0.079219,0.792450,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.261090


In [15]:
X_cv = X_train.copy()
X_cv['connected']=y_train

In [17]:
max_depth_list = np.linspace(1,10, num = 10, dtype=int)
num_features = np.linspace(1,20, num = 20, dtype = int)

accuracies = {}
for depth in max_depth_list:
    for num in num_features:
        fold_accuracy = 0
        for fold in [1,2,3,4,5]:      
            #Making  fold
            random.seed(fold)
            X_train_fold, X_val_fold, y_train_fold, y_val_fold = splitter(X_cv)
            ros = RandomOverSampler(random_state=0, sampling_strategy = 'minority')
            X_train_fold, y_train_fold = ros.fit_resample(
                X_train_fold, y_train_fold)  
            X_train_fold = X_train_fold.select_dtypes('number')
            X_val_fold = X_val_fold.select_dtypes('number')    
                
            RF = RandomForestClassifier(max_depth=depth, max_features = num) 
            RF.fit(X_train_fold.drop(columns = ["ID","pre_nucleus_id","post_nucleus_id"]),y_train_fold)

            y_hat_valid = RF.predict(X_val_fold.drop(columns = ["ID","pre_nucleus_id","post_nucleus_id"]))
            valid_acc = balanced_accuracy_score(y_val_fold, y_hat_valid)
            fold_accuracy += valid_acc
            print(f"depth: {depth}, num features {num}, valid accuracy for this fold, {valid_acc}")
        avg_fold_accuracy = fold_accuracy/5
        print(f"avgfold accuracy: {avg_fold_accuracy}")
        accuracies[(depth, num)] = avg_fold_accuracy
    
    

depth: 1, num features 1, valid accuracy for this fold, 0.7077811193737847
depth: 1, num features 1, valid accuracy for this fold, 0.6358232722900181
depth: 1, num features 1, valid accuracy for this fold, 0.6246528382646125
depth: 1, num features 1, valid accuracy for this fold, 0.6514211385713891
depth: 1, num features 1, valid accuracy for this fold, 0.5510814739414487
avgfold accuracy: 0.6341519684882506
depth: 1, num features 2, valid accuracy for this fold, 0.7191217390042561
depth: 1, num features 2, valid accuracy for this fold, 0.6416056865432078
depth: 1, num features 2, valid accuracy for this fold, 0.716640170163525
depth: 1, num features 2, valid accuracy for this fold, 0.749192796981522
depth: 1, num features 2, valid accuracy for this fold, 0.585409409476809
avgfold accuracy: 0.6823939604338639
depth: 1, num features 3, valid accuracy for this fold, 0.7442647054408555
depth: 1, num features 3, valid accuracy for this fold, 0.740610505372209
depth: 1, num features 3, vali

In [18]:
accuracy_df = pd.DataFrame({'Param Pair':accuracies.keys(), 'Avg Accuracy':accuracies.values()}).sort_values(by='Avg Accuracy', ascending=False).reset_index()
accuracy_df

,index,Param Pair,Avg Accuracy
0,88,"(5, 9)",0.773793
1,109,"(6, 10)",0.773017
2,87,"(5, 8)",0.771897
3,108,"(6, 9)",0.770698
4,107,"(6, 8)",0.769654
...,...,...,...
195,161,"(9, 2)",0.629723
196,182,"(10, 3)",0.606588
197,181,"(10, 2)",0.582383
198,160,"(9, 1)",0.573699


In [19]:
accuracy_df.to_csv('rf_depth_feat_cv.csv')

In [20]:
best_param = accuracy_df['Param Pair'][0]

In [21]:
X_train=X_train.select_dtypes('number')
ros = RandomOverSampler(random_state=0, sampling_strategy = 'minority')
X_train_use, y_train_use = ros.fit_resample(
                    X_train, y_train) 
X_val = X_val.select_dtypes('number')

In [22]:
xg_model = RandomForestClassifier(max_depth=best_param[0], max_features=best_param[1])
xg_model.fit(X_train_use.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1), y_train_use)

preds = xg_model.predict(X_val.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1))
balanced_accuracy_score(y_val, preds)

0.7574660490670425

In [24]:
best_features = pd.DataFrame({"Features":X_train_use.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).columns, 
              "Importances":abs(xg_model.feature_importances_)}).sort_values(by='Importances', ascending=False)
best_features

,Features,Importances
6,adp_dist,0.602951
26,RL_pre,0.081229
19,pre_nucleus_z,0.035886
14,post_test_score,0.035169
8,pre_skeletal_distance_to_soma,0.030032
11,pre_rf_x,0.027771
18,pre_nucleus_y,0.024377
21,post_nucleus_y,0.022498
28,V1_pre,0.014420
29,RL_post,0.013920


In [25]:
leaderboard_path = "../Data/leaderboard_data.csv"
sub_data = cleaner(leaderboard_path, feature_path, morph_path, submission = True)

In [26]:
sub_data = sub_data.select_dtypes('number')
sub_data.columns

Index(['ID', 'axonal_coor_x', 'axonal_coor_y', 'axonal_coor_z',
       'dendritic_coor_x', 'dendritic_coor_y', 'dendritic_coor_z', 'adp_dist',
       'post_skeletal_distance_to_soma', 'pre_skeletal_distance_to_soma',
       'pre_oracle', 'pre_test_score', 'pre_rf_x', 'pre_rf_y', 'post_oracle',
       'post_test_score', 'post_rf_x', 'post_rf_y', 'pre_nucleus_x',
       'pre_nucleus_y', 'pre_nucleus_z', 'post_nucleus_x', 'post_nucleus_y',
       'post_nucleus_z', 'pre_nucleus_id', 'post_nucleus_id', 'me_similarity',
       'fw_similarity', 'rf_similarity', 'AL_pre', 'RL_pre', 'V1_pre',
       'V1_post', 'RL_post', 'AL_post', 'area1', 'area2', 'area3',
       'minicol_dist', 'nuclei_adp_dist'],
      dtype='object')

In [27]:
preds = xg_model.predict(sub_data.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1))


In [28]:
sub_data['connected'] = preds==1

In [29]:
submission_data = sub_data.filter(['ID','connected'])
submission_data.to_csv('submission_data_rf.csv',index=False)